### Código para limpiar sensores 

In [2]:
import pandas as pd

df = pd.read_csv('Sensores_limpio_corregido.csv')  

df_sin_bateria = df[~df['entity_id'].str.contains('_battery', na=False)]

# Eliminamos filas que contienen un sensor nuevo que solo hace un registro 3 veces
df_sin_bateria = df_sin_bateria.drop(df[df['entity_id'].isin(['sensor.sensor_temperatura_3_nuevo_4_humidity','sensor.sensor_temperatura_3_nuevo_4_pressure','sensor.sensor_temperatura_3_nuevo_4_temperature'])].index)


sensores_ventanas_y_puerta = df_sin_bateria[
    (df_sin_bateria['entity_id'].str.contains('sensor.sensor_ventana_', na=False)) | 
    (df_sin_bateria['entity_id'] == 'sensor.sensor_puerta_1_device_temperature')
]

sensores_temperatura = df_sin_bateria[df_sin_bateria['entity_id'].str.contains('sensor.sensor_temperatura_', na=False)]

sensores_ventanas_y_puerta.to_csv('Sensores_ventanas_y_puerta.csv', index=False)
sensores_temperatura.to_csv('Sensores_temperatura_clase.csv', index=False)


### Código para limpiar csv de sol, quedando solo las variables de interés para predecir la temperatura del aula

In [3]:
df = pd.read_csv('CSV_datos_Sol_limpio.csv')  

df = df[["time", "azimuth", "elevation"]]

df.to_csv('Datos_sol_variables_importantes.csv', index=False)

### Código para limpiar csv de forecast

In [17]:
import json


df_raw = pd.read_csv("./entidad_weather_forecast.csv", sep=';', parse_dates=['time'])

# Si tiene columna 'attributes', expandirla
if 'attributes' in df_raw.columns:
    df_raw['attributes'] = df_raw['attributes'].apply(json.loads)
    atributos_expandidos = pd.json_normalize(df_raw['attributes'])
    df_limpio = pd.concat([df_raw.drop(columns=['attributes']), atributos_expandidos], axis=1)
else:
    df_limpio = df_raw


# Guardar el archivo limpio como CSV
df_limpio.to_csv("./forecast_limpio.csv", index=False)


df_silver = df_limpio[df_limpio['entity_id'] != "weather.forecast_casa"]
df_silver = df_silver[df_silver['state'] != 'unavailable']

# Comprobamos que la temperatura siempre está medida en grados Celsius
df_silver["temperature_unit"].unique()

# Nos quedamos con las columnas que nos interesan
df_silver = df_silver[["time", "temperature", "cloud_coverage"]]
df_silver['time'] = pd.to_datetime(df_silver['time']).dt.strftime('%Y-%m-%d %H:%M:%S')

df_silver.to_csv('pronostico_exterior.csv', index=False)
